In [1]:
import rasterio
from rasterio.plot import show
import json
import pandas as pd
import numpy as np
from haversine import haversine

In [19]:
class DensityExtractor():
    def __init__(self, data_path: str):
        self._path_to_csv_data = data_path

    @property # getter
    def csv_data(self):
        return self._csv_data

    def load_csv_data(self) -> pd.DataFrame:
        csv_data = pd.read_csv(self._path_to_csv_data)
        csv_data = csv_data.rename(
            columns=
            {
                "X": "longitude",
                "Y": "latitude",
                "Z": "population_density_km2"
            }
        )
        self._csv_data = csv_data
    
    def get_min_distance_density(self, latitude: float, longitude: float) -> float:
        coord_input = (latitude, longitude)
        dataset = self._csv_data

        dataset['distance_km'] = dataset.apply(
        lambda row: haversine(coord_input, (row['latitude'], row['longitude'])),
        axis=1)
        
        min_index = dataset['distance_km'].idxmin()
        nearest_point = dataset.iloc[[min_index]]
    
        return nearest_point['population_density_km2']



In [14]:
directory_density_path = "../config/cfg_density_population.json"

with open(directory_density_path) as f:
    data_json = json.load(f)

# Print the data (it will be stored as a Python dictionary)
density_population_path = data_json['path_csv']

dens_extractor = DensityExtractor(density_population_path)

In [20]:
dens_extractor.load_csv_data()
dens_extractor.csv_data

,longitude,latitude,population_density_km2
0,30.436250,-1.046250,358.407013
1,30.444583,-1.046250,610.205383
2,30.452917,-1.046250,1437.951172
3,30.461250,-1.046250,2122.332031
4,30.402917,-1.054583,187.413574
...,...,...,...
28399,29.377917,-2.821250,0.000000
28400,29.519583,-2.821250,414.417999
28401,29.527917,-2.821250,409.061432
28402,29.536250,-2.821250,416.264374


In [21]:
latitude = -2.253439903
longitude = 30.80427933

min_distance = dens_extractor.get_min_distance_density(latitude=latitude, longitude=longitude)

In [23]:
min_distance

20224    878.588684
Name: population_density_km2, dtype: float64

In [88]:
data = dens_extractor.csv_data 
data[(data["longitude"].round(6) == min_distance["longitude"].round(6)) & (data["latitude"].round(6) == min_distance["latitude"].round(6))]

,longitude,latitude,population_density_km2,distance_km
20224,30.802917,-2.254583,878.588684,0.197703


In [81]:
print(min_distance)

longitude              30.802917
latitude               -2.254583
population_density    878.588684
distance                0.197703
Name: 20224, dtype: float64


In [53]:


df_filtered = df[(df['latitude'].round(6) <= latitude) & (df['longitude'].round(6) <= longitude)]
df_filtered

,longitude,latitude,population_density
20028,29.161250,-2.254583,31.835442
20029,29.177917,-2.254583,27.245543
20030,29.186250,-2.254583,17.712193
20031,29.194583,-2.254583,119.946495
20032,29.202917,-2.254583,151.979599
...,...,...,...
28399,29.377917,-2.821250,0.000000
28400,29.519583,-2.821250,414.417999
28401,29.527917,-2.821250,409.061432
28402,29.536250,-2.821250,416.264374


In [45]:
df_prova = df[np.isclose(df['longitude'], 30.436250)] # funziona
df_prova = df[df['longitude'].round(6) == 30.436250] # funziona
df_prova

,longitude,latitude,population_density
0,30.43625,-1.046250,358.407013
7,30.43625,-1.054583,370.727692
23,30.43625,-1.062916,346.225922
38,30.43625,-1.071250,278.492615
52,30.43625,-1.079583,292.766083
...,...,...,...
21000,30.43625,-2.287916,242.237976
21210,30.43625,-2.296250,264.536957
21425,30.43625,-2.304583,279.894257
21637,30.43625,-2.312916,276.680603
